<a id="top"></a>
# Pneumonia Classification Sample Application

## OpenVINO version check:
You are currently using the Intel® Distribution of OpenVINO™ Toolkit latest version. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by [clicking this link](../../../../openvino-lts/developer-samples/python/pneumonia-classification/classification_pneumonia.ipynb).

## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    - **classification_pneumonia.ipynb** - This Jupyter* Notebook
    - **classification_pneumonia.py** - Python* code for pneumonia classification application
    - **model.pb** - TensorFlow trained model
    - **utils.py** - Utility Python* code to read and write image results
    - **utils_image.py** - Utility Python* code to display results
    - **validation_images/** - Test image dataset
    - **example_CAM.png**, **example-pneumonia.jpeg** - Images used in this Jupyter* Notebook

It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter* Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>


## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform pneumonia classification.  This solution uses an inference model that has been trained to classify the presence of pneumonia using a patient's chest X-ray.  The results are visualized from what the network has learned using the Class Activation Maps (CAM) technique.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Image input is supported using OpenCV
  - Visualization of the resulting class activation maps
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) and [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)
  - Access and use of a class activation map as part of inference

### Application background 
Pneumonia is an inflammatory condition of the lung affecting primarily the small air sacs known as alveoli.
Typically symptoms include some combination of cough, chest pain, fever, and trouble breathing [1].
Pneumonia affects approximately 450 million people globally (7% of the population) and results in about 4 million deaths per year [2]. To diagnose this disease, chest X-ray images remain the best diagnostic tool.

![](example-pneumonia.jpeg) *Chest X-ray image of a patient with Pneumonia*

In this application, we use a model trained to classify patients with pneumonia over healthy cases based on their chest X-ray images. The topology used is the DenseNet 121 which is an architecture that has shown to be very efficient at this problem.  DenseNet 121 is the first work to claim a classification rate better than practicing radiologists. The dataset used for training is from the "Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification" [3] with a [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/).
The trained model is provided as a frozen Tensorflow model.

[1] Ashby B, Turkington C (2007). The encyclopedia of infectious diseases (3rd ed.). New York: Facts on File. p. 242. ISBN 978-0-8160-6397-0. 

[2] Ruuskanen O, Lahti E, Jennings LC, Murdoch DR (April 2011). "Viral pneumonia". Lancet. 377 (9773): 1264–75. 

[3] [Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification](https://data.mendeley.com/datasets/rscbjbr9sj/2) [(Direct link to dataset)](http://dx.doi.org/10.17632/rscbjbr9sj.2#file-41d542e7-7f91-47f6-9ff2-dd8e5a5a7861)

### Classification and class activation maps

A Class Activation Map (CAM) [1] is a technique to visualize the regions that are relevant within a Convolutional Neural Network to identify the specific class in the image. 
The CAM $M(x,y)$ is calculated from the *N* feature maps $f_i(x,y)$ within the last convolutional layer of the network.  The weighted sum of those feature maps based on the weights of the fully connected layer $w_i$, represents how important those feature maps are for the given classification output. 

$ M(x,y)=\sum_{i=0}^N w_i f_i(x,y) $

When using the Intel® Distribution of OpenVINO™ toolkit, this can be implemented using the Python* code (Found in [classification_pneumonia.py](./classification_pneumonia.py)):

```python
def class_activation_map_openvino(res, convb, fc, net, fp16):
    res_bn = res[convb]
    conv_outputs=res_bn[0,:,:,:]
    # retrieve layer weights
    node_fc=get_ngraph_node_by_name(net,fc)
    weights_fc=node_fc.get_data()[0,:]   
    # initialize CAM array
    cam = np.zeros(dtype=np.float32, shape=conv_outputs.shape[1:])
    # perform weighted sum
    for i, w in enumerate(weights_fc):
        conv_outputs1=conv_outputs[i, :, :]
        if fp16:
            w=float16_conversion(w)
            conv_outputs1=float16_conversion_array(conv_outputs[i, :, :])
        cam += w * conv_outputs1
    return cam
```

To calculate the Class Activation Map, we access the output feature maps of the last convolution layer `convb` and the weights of the fully connected layer `fc`. 
By default, only the output layer is present in the returned output of the network. In order to get the feature maps, the last convolution layer must be added to the output. 
This is done by using the function [`add_outputs(`*convb*`)`](https://docs.openvinotoolkit.org/latest/ie_python_api/classie__api_1_1IENetwork.html) on the network before loading it to the the device plugin. 
In order to obtain the layer's name, we recommend [Netron](https://pypi.org/project/netron/), which allows to visualize graphically the model. 

In our pneumonia classification model, the name of the last convolutional layer `convb` is "relu_1/Relu" and the name of the fully-connected layer `fc` is "predictions_1/MatMul".


In order to get the feature maps, inference must be performed first to get the results. The `res` argument in the code above is the inference output which includes the feature maps by using `add_outputs()`.
We access the `fc` layer weights using the call [`net.layers.get(fc).blobs["weights"]`](https://docs.openvinotoolkit.org/latest/ie_python_api/classie__api_1_1IENetLayer.html) on the network `net`.  Before doing the weighted sum, if the FP16 model is being used, the weights and feature maps value must be converted from type `float16`to type `float` using the function `float16_conversion_array()`. The weighted sum of the weights is then done with the feature maps. 

The result of the function is an image of same size as the feature maps (here, $7\times7$).  An example is shown below on the left.
The CAM image is upsampled to the original input image size and overlaid over the input image. The region of highest value (here, in yellow) will be the region that is the most relevant to deciding on the classification result value. 

![](example_CAM.png) *CAM image and overlay of CAM with input image*

[1] Zhou, Bolei, et al. "Learning deep features for discriminative localization." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

## Pneumonia classification application
The pneumonia classification application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on an input X-ray image to classify the presence of pneumonia.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the pneumonia classification application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified input X-ray images.  The result for each input X-ray image is written to the `results.txt` file in the form:
   Pneumonia probability: <*% probability*>, Inference performed in <*time*>, Input file: <*input filename*>

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU and Intel® Movidius™ Neural Compute Stick 2.  After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook using the `show_results` utility.

The application and inference code for this sample is already implemented in the Python* files: [`classification_pneumonia.py`](./classification_pneumonia.py), [`utils.py`](./utils.py), and [`utils_image.py`](./utils_image.py).

The following sections will guide you through configuring and running the pneumonia classification application.

### Configuration
The following sections describe all the necessary configuration to run the pneumonia classification application.
#### Command line arguments
The application is run from the command line using the following format:
```bash
python3 classification_pneumonia.py <arguments...>
```
The required command line _<arguments...>_ to run the Python* executable [`classification_pneumonia.py`](./classification_pneumonia.py) are:
- **-m** - Path to the _.xml_ IR file (created using [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)) for the inference model. 
- **-i** - Path to input X-ray image file(s) (may use wildcard characters '?' and '*')
- **-o** - The path to where the output video file will be stored
- **-d** - Device type to use to run inference (CPU, GPU, MYRIAD, HDDL)

### Imports
We begin by importing all the Python* modules that will be used within this Jupyter* Notebook to run and display the results of the pneumonia classification application on the Intel® DevCloud for the Edge:
- [os](https://docs.python.org/3/library/os.html#module-os) - Operating system specific module (used for file name parsing)
- [time](https://docs.python.org/3/library/time.html#module-time) - Time tracking module (used for measuring execution time)
- [matplotlib.pyplot](https://matplotlib.org/) - pyplot is used for displaying output images
- [sys](https://docs.python.org/3/library/sys.html#module-sys) - System specific parameters and functions
- [qarpo.demoutils](https://github.com/ColfaxResearch/qarpo) - Provides utilities for displaying results and managing jobs from within this Jupyter* Notebook
- [utils_image](./utils_image.py) - Provides show_results() utility specific to this sample for displaying the results

Run the following cell to import the Python* dependencies needed.

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [ ]:
import matplotlib.pyplot as plt
import os
import time
import sys
from qarpo.demoutils import *
from utils_image import show_results
print('Imported Python modules successfully.')

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.).  For this sample, the model is supplied in the file `model.pb` 

The `model.pb` file will need to be optimized using the Model Optimizer to create the necessary IR files.  We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the two most common precisions, FP32 and FP16.

For this model, we will run the Model Optimizer using the format:
```bash
!mo.py \
    --input_model <path_to_model> \
    --input_shape=[<N,H,W,C>] \
    --data_type <data_precision> \
    -o <path_to_output_directory> \
    --mean_values [<channel_mean_values>] \
    --scale_values [<scale_values>]
```

The input arguments are as follows:
- **--input_model** : The model's input *.pb* file
- **--input_shape** : The model's input data shape
 - **N** = Batch size
 - **H** = Height
 - **W** = Width
 - **C** = Number of channels
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--o** : Output directory where to store the generated IR model files
- **--scale_values** : Scaling (divide by, one per channel) value to apply to input values
- **--mean_values** : Mean values (one per channel) to be subtracted from input values before scaling

The input shape, scaling values, and mean values we will be using are specific to the model topology being used.  Using the appropriate values for the model that we will use, The complete command will look like the following:
```bash
!mo.py \
    --input_model model/crnn.pb \
    --input_shape=[1,224,224,3] \
    --data_type <data_precision> \
    -o models/<data_precision> \
    --mean_values [123.75,116.28,103.58] \
    --scale_values [58.395,57.12,57.375]
```
We will run the command twice, first with <*data_precision*> set to `FP16` and then `FP32` to get all the IR files we will need to run inference on different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert TensorFlow* models as well as specifying input shape and scaling parameters for common model topologies may be found at:[Converting a TensorFlow* Model](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html)
</i></div>


Run the following cell to use the Model Optimizer to create the `FP16` and `FP32` model IR files:

In [ ]:
# Create FP16 IR files
!mo.py \
--input_model model.pb \
--input_shape=[1,224,224,3] \
--data_type FP16 \
-o models/FP16/ \
--mean_values [123.75,116.28,103.58] \
--scale_values [58.395,57.12,57.375]

# Create FP32 IR files
!mo.py \
--input_model model.pb \
--input_shape=[1,224,224,3] \
--data_type FP32 \
-o models/FP32 \
--mean_values [123.75,116.28,103.58] \
--scale_values [58.395,57.12,57.375] 

# find all resulting IR files
!echo "\nAll IR files that were downloaded or created:"
!find ./models -name "*.xml" -o -name "*.bin"

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning of a line is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

As shown above from the output of the last `!find...` command, the required sets of IR model files (`*.xml` and `*.bin`) have been created.  

### Run inference
The following sections will go through the steps to run our inference application on the Intel® DevCloud for the Edge. 

#### Configure input
For convenience and consistency, in the next cell we set the Python* variable `InputImages` to the input image file(s) that we will be using to run our sample application.

<br><div class=note><i><b>Note: </b>
If you want to use different input images, change the path in the following cell to the path of the images and run the cell again.
</i></div>

In [ ]:
# Set the path to the input images to use for the rest of this sample
InputImages = 'validation_images/PNEUMONIA/*.jpeg'
print(f"Input image files set to:{InputImages}")

#### Optional exercise: View input images without inference

If you are curious to see the input images, run the following cell to view the original video stream used for inference and this sample.

In [ ]:
import glob
from IPython.display import Image, display
# find all the input images
files=glob.glob(InputImages)
# display input images
for image_file in files:
    print(f"Input image file {image_file}:")
    display(Image(image_file))


#### Create the job file
We will run inference on several different edge compute nodes present in the Intel® DevCloud for the Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that will be executed directly on the edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
classification_pneumonia_job.sh <output_directory> <device> <fp_precision> <input_file> 
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*device*> - Hardware device to use (e.g. CPU, GPU, etc.)
- <*fp_precision*> - Which floating point precision inference model to use (FP32 or FP16)
- <*input_file*> - Path to input image file(s)

Based on the input arguments, the job file will do the following:
- Change to the working directory `PBS_O_WORKDIR` where this Jupyter* Notebook and other files appear on the compute node
- Create the <*output_directory*>
- Choose the appropriate inference model IR file for the specified <*fp_precision*>
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `classification_pneumonia_job.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `classification_pneumonia_job.sh`.

In [ ]:
%%writefile classification_pneumonia_job.sh

# Store input arguments: <output_directory> <device> <fp_precision> <input_file>
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE="$4"

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Set inference model IR files using specified precision
MODELPATH=models/${FP_MODEL}/model.xml
pip3 install Pillow
# Run the pneumonia detection code
python3 classification_pneumonia.py -m $MODELPATH \
                                    -i "$INPUT_FILE" \
                                    -o $OUTPUT_FILE \
                                    -d $DEVICE

#### How to submit a job

Now that we have the job script, we can submit jobs to edge compute nodes in the Intel® DevCloud for the Edge.  To submit a job, the `qsub` command is used with the following format:
```bash
qsub <job_file> -N <JobName> -l <nodes> -F "<job_file_arguments>" 
```
We can submit classification_pneumonia_job.sh to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- <*job_file*> - This is the job file we created in the previous step
- `-N` <*JobName*> : Sets name specific to the job so that it is easier to distinguish  between it and other jobs
- `-l` <*nodes*> - Specifies the number and the type of nodes using the format *nodes*=<*node_count*>:<*property*>[:<*property*>...]
- `-F` "<*job_file_arguments*>" - String containing the input arguments described in the previous step to use when running the job file

*(Optional)*: To see the available types of nodes on the Intel® DevCloud for the Edge, run the following cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

In the above output from executing the previous cell, the properties describe the node, and the number on the left is the number of available nodes of that architecture.

#### Submit jobs

Each of the cells in the subsections below will submit a job to be run on different edge compute nodes. The output of each cell is the _JobID_ for the submitted job.  The _JobID_ can be used to track the status of the job.  After submission, a job will go into a waiting queue before running once the requested compute nodes become available.

<br><div class=note><i><b>Note: </b>You may submit all jobs at once or one at a time.</i></div> 

<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>

##### Submit to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an edge node with an [Intel® Core™ i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) processor. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core = !qsub classification_pneumonia_job.sh -l nodes=1:idc001skl -F "results/core/ CPU FP32 \"{InputImages}\"" -N pneum_core
print(job_id_core[0])
#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® Gold 6258R CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Gold 6258R Processor](https://ark.intel.com/content/www/us/en/ark/products/199350/intel-xeon-gold-6258r-processor-38-5m-cache-2-70-ghz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_cascade_lake = !qsub classification_pneumonia_job.sh  -l nodes=1:idc018 -F "results/xeon_cascade_lake/ CPU FP32 \"{InputImages}\"" -N pneum_xeon_cascade_lake 
print(job_id_xeon_cascade_lake[0])
#Progress indicators
if job_id_xeon_cascade_lake:
    progressIndicator('results/xeon_cascade_lake', 'i_progress_'+job_id_xeon_cascade_lake[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® E3-1268L V5 CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Processor E3-1268L v5](https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_skylake = !qsub classification_pneumonia_job.sh  -l nodes=1:idc007xv5 -F "results/xeon_skylake/ CPU FP32 \"{InputImages}\"" -N pneum_xeon_skylake 
print(job_id_xeon_skylake[0])
#Progress indicators
if job_id_xeon_skylake:
    progressIndicator('results/xeon_skylake', 'i_progress_'+job_id_xeon_skylake[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Core CPU and using the integrated Intel® GPU
In the cell below, we submit a job to an edge node with an [Intel® Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on the Intel® HD Graphics 530 GPU integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub classification_pneumonia_job.sh -l nodes=1:idc001skl -F "results/gpu/ GPU FP16 \"{InputImages}\"" -N pneum_gpu 
print(job_id_gpu[0])
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Neural Compute Stick 2
In the cell below, we submit a job to an edge node with an [Intel Core i5-6500te](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) CPU. The inference workload will run on an [Intel® Neural Compute Stick 2](https://software.intel.com/en-us/neural-compute-stick) installed in this node.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub classification_pneumonia_job.sh -l nodes=1:idc004nc2 -F "results/ncs2/ MYRIAD FP16 \"{InputImages}\"" -N pneum_ncs2
print(job_id_ncs2[0])
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Atom® and using the integrated Intel® GPU
In the cell below, we submit a job to an edge node with an [Intel® Atom® x7-E3950](https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz.html) processor. The inference workload will run on the integrated Intel® HD Graphics 505 GPU.

In [ ]:
#Submit job to the queue
job_id_atom = !qsub classification_pneumonia_job.sh -l nodes=1:idc008u2g -F "results/atom/ GPU FP16 \"{InputImages}\"" -N pneum_atom
print(job_id_atom[0]) 
#Progress indicators
if job_id_atom:
    progressIndicator('results/atom', 'i_progress_'+job_id_atom[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Vision Accelerator Design with Intel® Movidius™ VPUs
In the cell below, we submit a job to an edge node with an [Intel Core i5-6500te CPU](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on an [Intel® Vision Accelerator Design with Intel® Movidius™ VPUs](https://software.intel.com/openvino-toolkit/hardware#VPU) accelerator installed in this node that has eight VPUs.

In [ ]:
#Submit job to the queue
job_id_hddlr = !qsub classification_pneumonia_job.sh -l nodes=1:idc002mx8 -F "results/hddlr/ HDDL FP16 \"{InputImages}\"" -N pneum_hddlr
print(job_id_hddlr[0])
#Progress indicators
if job_id_hddlr:
    progressIndicator('results/hddlr', 'i_progress_'+job_id_hddlr[0]+'.txt', "Inference", 0, 100)

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 

In [ ]:
liveQstat()

You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If ste status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

<br><div class=danger><b>Wait!: </b>Please wait for the inference jobs and video rendering to complete before proceeding to the next step to view results.</div>

### View results

Once the jobs have completed, the queue system outputs the stdout and stderr streams of each job into files with names of the forms <*JobName*>.o<*JobID*> and <*JobName*>.e<*JobID*>, respecitvely.  The *JobName* corresponds to the `-N` option when submitting the job using the `qsub` command.  

The output for each job is written to the files `result_{job_id}.txt` and `result_<image_nm>.jpeg` located in the directory `results/<device>` that was specified as the output directory to the job file.  We will now use the utility `show_results()` utility to display the output within this Jupyter* notebook.  Calling `show_results()` from a Python* cell follows the form:
```python
show_results(result_dir)
```
The parameters are:
- *result_dir* - Path to output directory where the output files `result.txt` and `result_<image_nm>.jpeg` are located

Run the cells below to display the results.

<br><div class=note><i><b>Note: </b>See [`utils_image.py`](./utils_image.py) for more information on how the results are displayed in Jupyter* Notebooks.</i></div>

##### View results from an Intel® CPU

In [ ]:
print('Intel Core CPU Results:')
show_results('results/core', f"result_{job_id_core[0]}.txt")

##### View results from an Intel® Xeon® Gold 6258R CPU

In [ ]:
print('Intel Xeon Cascade Lake CPU Results:')
show_results('results/xeon_cascade_lake', f"result_{job_id_xeon_cascade_lake[0]}.txt")

##### View results from an Intel® Xeon® E3-1268L v5 CPU

In [ ]:
print('Intel Xeon Skylake CPU Results:')
show_results('results/xeon_skylake', f"result_{job_id_xeon_skylake[0]}.txt")

##### View results from an Intel® Core CPU and integrated Intel® GPU

In [ ]:
print('Intel Core + Intel GPU Results:')
show_results('results/gpu', f"result_{job_id_gpu[0]}.txt")

##### View results from an Intel® Neural Compute Stick 2

In [ ]:
print('Intel CPU + Intel NCS2 Results:')
show_results('results/ncs2', f"result_{job_id_ncs2[0]}.txt")

##### View results from an Intel® Atom® and integrated Intel® GPU

In [ ]:
print('Intel Atom CPU + Intel GPU Results:')
show_results('results/atom', f"result_{job_id_atom[0]}.txt")

##### View results from Intel® Vision Accelerator Design with Intel® Movidius™ VPUs

In [ ]:
print('Intel VPU HDDL-R Results:')
show_results('results/hddlr', f"result_{job_id_hddlr[0]}.txt")

### View and assess performance results

The running time of each inference task is recorded in `results/<device>/stats.txt`. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance for **Inference Engine Processing Time** and higher values mean better performance for **Inference Engine FPS**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

In [ ]:
# Create table of <architecture>, <title> for plotting
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon_cascade_lake', 'Intel Xeon\nGold\n 6258R\nCPU'),
             ('xeon_skylake', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nHD530/GPU'),
             ('ncs2', 'Intel VPU\ni5-6500TE\nNCS2 VPU'),
             ('atom', 'Intel Atom\nx7-E3950\nHD505/GPU'),
             ('hddlr', 'Intel VPU\n8xVPU\nHDDL-R')]
# For each archtecture in table, create path to stats file or placeholder 
stats_list = []
for arch, a_name in arch_list:
    # if job_id_<architecture> exists, the job was run and has a stats file
    if 'job_id_'+arch in vars():
        stats_list.append((f'results/{arch}/stats_{vars()["job_id_"+arch][0]}.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
# Plot the execution time from the stats files
summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time')
# Plot the frames per second from the stats files
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps')

## Telemetry Dashboard
Once your submitted jobs are completed, run the cells below to view telemetry dashboards containing performance metrics for your model and target architecture.

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core™ i5-6500TE</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_core[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® Gold 6258R CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_cascade_lake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® E3-1268L v5 CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_skylake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Core CPU and using the onboard Intel® GPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_gpu[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on IEI Mustang-V100-MX8 ( Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_hddlr[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® NCS 2 (Neural Compute Stick 2)</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_ncs2[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on UP Squared Grove IoT Development Kit (UP2)</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_atom[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>